In [ ]:
import os, time, re
import numpy as np
import pandas as pd
import multiprocessing

from multiprocessing import Pool
from itertools import product
from multi_process import apply_f, merge_cases, init_process

In [ ]:
def get_max_charge(charge_grid_level):
    
    charge = -1
    if type(charge_grid_level) == int:
        charge = charge_grid_level
        
    elif type(charge_grid_level) == str:
        # like '2,3,4'
        if len(charge_grid_level) > 2:
            charge = max(int(x) for x in charge_grid_level.split(','))
        else: # like *, empty string
            charge = charge_grid_level
    else:
        charge = ''
    
    return charge

In [ ]:
file = './data/CPCS TRIS Source Data/Priority3-ChargesDispositions-Full.xlsx'
data = pd.read_excel(file, sheet_name='Data', skiprows=4)
data

In [ ]:
print(len(data.columns))
data['max_charge_grid_level'] = data['charge_grid_level'].apply(get_max_charge)
print(len(data.columns))

In [ ]:
cols = list(data.columns)
cols.remove('max_charge_grid_level')
i = cols.index('charge_grid_level')
cols.insert(i+1, 'max_charge_grid_level')

data[cols].to_excel('./data/CPCS TRIS Source Data/Priority3-MaxChargesDispositions-Full-2.xlsx', encoding='utf8', index=False)

In [ ]:
case_ids = list(set(data['CASE_ID']))
case_ids[:5]

In [ ]:
group = data.groupby(by=['CASE_ID'])
group

In [ ]:
group.get_group(45).columns.to_list()

In [ ]:
part_case_ids = np.array_split(case_ids, multiprocessing.cpu_count())
print('start')
start = time.time()
with Pool(processes=multiprocessing.cpu_count(), initializer=init_process, initargs=(None,)) as pool:
    result_parts = pool.starmap(apply_f, product([merge_cases], part_case_ids, [data], [group]))
print("time cost={}".format(time.time() - start))
start = time.time()
tmp_data = pd.concat(result_parts)
print("time cost={}".format(time.time() - start))

In [ ]:
err = sorted(set(case_ids) - set(tmp_data['CASE_ID']))
print(f'missing caseID: {len(err)}\n{err}')

tmp_data

In [ ]:
# tmp_data.to_excel('./data/CPCS TRIS Source Data/Priority3-MaxChargesDispositions-Full.xlsx', encoding='utf8', index=False)
tmp_data = pd.read_excel('./data/CPCS TRIS Source Data/Priority3-MaxChargesDispositions-Full-0.xlsx')
tmp_data

In [ ]:
tmp = tmp_data[['CASE_ID', 'charge_grid_level']]
tmp = tmp.rename(columns={'charge_grid_level': 'max_charge_grid_level'})
tmp

In [ ]:
new = data.merge(tmp, how='left', on='CASE_ID')
print(len(new), len(data))
new

In [ ]:
cols = list(data.columns)
i = cols.index('charge_grid_level')
cols.insert(i+1, 'max_charge_grid_level')

new[cols].to_excel('./data/CPCS TRIS Source Data/Priority3-MaxChargesDispositions-Full-1.xlsx', encoding='utf8', index=False)

In [ ]:
# this cost 290 secs, too slow with single process, use multi-processing instead
tmp_data = pd.DataFrame(columns=data.columns)
err = []
start = time.time()

for case_id in case_ids[:]:
    charge = -1
    max_charge = -1
    index = -1
    case_charges = group.get_group(case_id)['charge_grid_level']
    for i in case_charges.index:
        charge_grid_level = case_charges[i]
        if type(charge_grid_level) == int:
            charge = charge_grid_level
        
        # like '2,3,4'
        elif type(charge_grid_level) == str and len(charge_grid_level) > 2:
            charge = max(int(x) for x in charge_grid_level.split(','))
        
        # else is NaN, type = float
#         print(case_id, i, 'if else', index, max_charge, charge)
        if max_charge < charge:
            max_charge = charge
            index = i
#         print(case_id, i, 'if else', index, max_charge, charge)
#     print(index, max_charge)
    if index > -1:
        row = data.loc[index].copy()
        row.loc['charge_grid_level'] = max_charge
        tmp_data = tmp_data.append(row, ignore_index=True)
    else:
        err.append(case_id)

print(f'time: {time.time() - start}')
print(f'err: {len(err)}\n{err}')
tmp_data

In [ ]:
len(err), len(data)

In [ ]:
tmp_data[tmp_data['CASE_ID']==49]